In [1]:
library(RPostgreSQL)
library(tableone)
library(tidyverse)

Loading required package: DBI
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
sql <-
"
select
sum(case when extract(day from (deathtime - intime)) < 1 then 1 else 0 end) as day1,
sum(case when extract(day from (deathtime - intime)) < 2 then 1 else 0 end) as day2
from merged_data
"

dbGetQuery(con, sql)

day1,day2
216,363


In [5]:
sql <-
"
with stg_0 as (
    select co.icustay_id, co.deathtime, co.intime, fl.day2, fl.day3
    from merged_data co left join fluid fl on co.icustay_id = fl.icustay_id
)

select
sum(case when extract(day from (deathtime - intime)) < 1 and day2 is not null then 1 else 0 end) as day2,
sum(case when extract(day from (deathtime - intime)) < 2 and day3 is not null then 1 else 0 end) as day3
from stg_0
"

dbGetQuery(con, sql)

day2,day3
8,8


In [6]:
sql <-
"
with stg_0 as (
    select co.icustay_id, co.deathtime, co.intime, fl.day1, fl.day2, fl.day3
    from merged_data co left join fluid fl on co.icustay_id = fl.icustay_id
)

select
sum(case when day1 is null then 1 else 0 end) as day1,
sum(case when day2 is null then 1 else 0 end) as day2,
sum(case when day3 is null then 1 else 0 end) as day3
from stg_0
"

dbGetQuery(con, sql)

day1,day2,day3
230,976,2100


In [7]:
sql <-
"
select count(*) from lactate_death
"

dbGetQuery(con, sql)

count
433


In [8]:
sql <-
"
with stg_0 as (
    select co.icustay_id, sr.lactate_diff
    from merged_data co left join lactate_diff sr on co.hadm_id = sr.hadm_id
)

select
sum(case when lactate_diff is null then 1 else 0 end) as lactate_diff
from stg_0
"

dbGetQuery(con, sql)

lactate_diff
4555


In [9]:
sql <-
"
with stg_0 as (
    select co.hadm_id, sr.lactate_diff
    from merged_data co left join lactate_diff sr on co.hadm_id = sr.hadm_id
)

select
sum(case when b.death = 1 and a.lactate_diff is not null then 1 else 0 end) as lactate_diff
from stg_0 a
left join (select hadm_id, 1 as death from lactate_death) b on a.hadm_id = b.hadm_id
"

dbGetQuery(con, sql)

lactate_diff
1


In [10]:
sql <-
"
select count(*)
from lactate_diff inner join lactate_death using (hadm_id)
"

dbGetQuery(con, sql)

count
1


In [11]:
sql <-
"
select count(*) from creatinine_death
"

dbGetQuery(con, sql)

count
428


In [12]:
sql <-
"
with stg_0 as (
    select co.icustay_id, sr.creatinine_diff
    from merged_data co left join creatinine_diff sr on co.hadm_id = sr.hadm_id
)

select
sum(case when creatinine_diff is null then 1 else 0 end) as creatinine_diff
from stg_0
"

dbGetQuery(con, sql)

creatinine_diff
1141


In [13]:
sql <-
"
select count(*)
from creatinine_diff inner join creatinine_death using (hadm_id)
"

dbGetQuery(con, sql)

count
1


In [14]:
sql <-
"
with stg_0 as (
    select co.hadm_id, sr.creatinine_diff
    from merged_data co left join creatinine_diff sr on co.hadm_id = sr.hadm_id
)

select
sum(case when b.death = 1 and a.creatinine_diff is not null then 1 else 0 end) as creatinine_diff
from stg_0 a
left join (select hadm_id, 1 as death from creatinine_death) b on a.hadm_id = b.hadm_id
"

dbGetQuery(con, sql)

creatinine_diff
1


In [15]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE